# Project: Automated Fact Checking For Climate Science Claims

Student Name: Hongda Zhu

Student ID: 1259524

In [2]:
import sys
print(sys.version)

3.8.16 (default, Mar  1 2023, 21:19:10) 
[Clang 14.0.6 ]


# Load datasets

In [1]:
import json

def load_data(filename):
    file = open(filename, 'r')
    data = json.load(file)
    file.close()
    return data

train_data = load_data("train-claims.json")
dev_data = load_data("dev-claims.json")
evidence_data = load_data("evidence.json")
test_data = load_data("test-claims-unlabelled.json")

# print("train_data: ", train_data)
# print("dev_data: ", dev_data)
# print("evidence_data: ", evidence_data)
# for id, info in train_data.items():
#     print("claim_id: ", id)
#     print("claim_text: ", info['claim_text'])
#     print("claim_label: ", info['claim_label'])
#     print("evidences: ", info['evidences'])
#     print()
# print("dev_data: ", dev_data)
# print("evidence_data: ", evidence_data)

# Preprocessing the data

Evidence retrieval.

Retrieve the top k relevant evidences

In [2]:
import torch
from sklearn.feature_extraction.text import TfidfVectorizer

# Retrieve top-k evidence
def retrieve_top_k_evidence(claim, evidence_corpus, device, k=5):
    evidence_ids = list(evidence_corpus.keys())
    evidence_texts = list(evidence_corpus.values())
    
    vectorizer = TfidfVectorizer(stop_words='english')
    evidence_matrix = vectorizer.fit_transform(evidence_texts)
    claim_vector = vectorizer.transform([claim])
    
    # Convert to PyTorch tensors and move to GPU
    evidence_matrix_tensor = torch.tensor(evidence_matrix.toarray(), device=device, dtype=torch.float)
    claim_vector_tensor = torch.tensor(claim_vector.toarray(), device=device, dtype=torch.float)
    
    # Compute cosine similarity on GPU
    dot_product = torch.matmul(claim_vector_tensor, evidence_matrix_tensor.T)
    claim_norm = torch.norm(claim_vector_tensor, dim=1).unsqueeze(1)
    evidence_norm = torch.norm(evidence_matrix_tensor, dim=1).unsqueeze(0)
    cosine_similarity = dot_product / (claim_norm * evidence_norm)

    # Sort and retrieve top-k indices
    sorted_indices = torch.argsort(cosine_similarity, descending=True)
    top_k_indices = sorted_indices[0, :k].cpu().numpy()
    
    top_k_evidence_ids = [evidence_ids[i] for i in top_k_indices]
    return top_k_evidence_ids


Retrieve all relevant evidences

In [ ]:
import torch.nn.functional.cosine_similarity

def retrieve_all_relevant_evidences(claim, evidence_data):
    evidence_ids = list(evidence_data.keys())
    evidence_texts = list(evidence_data.values())

    vectorizer = TfidfVectorizer(stop_words='english')
    evidence_matrix = vectorizer.fit_transform(evidence_texts)
    claim_vector = vectorizer.transform([claim])

    similarities = cosine_similarity(claim_vector, evidence_matrix)
    relevant_indices = [i for i, sim in enumerate(similarities[0]) if sim > 0]

    relevant_evidence_ids = [evidence_ids[i] for i in relevant_indices]
    return relevant_evidence_ids


Create PyTorch Dataset and DataLoader for preprocessed data

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class convertPytorchDataset(Dataset):
    def __init__(self, claim_data, evidence_data, tokenizer, label_mapping):
        self.claim_data = claim_data
        self.evidence_data = evidence_data
        self.tokenizer = tokenizer
        self.label_mapping = label_mapping

    def __len__(self):
        return len(self.claim_data)

    def __getitem__(self, idx):
        claim_id, claim_data = list(self.claim_data.items())[idx]
        claim_text = claim_data["claim_text"]
        claim_label = claim_data.get("claim_label", None)

        # Retrieve all evidence for the claim
        evidence_ids = retrieve_top_k_evidence(claim_text, self.evidence_data, device=device, k=5)

        # Tokenize claim and evidence
        tokens = self.tokenizer.encode_plus(claim_text, truncation=True, padding="max_length", max_length=512)

        # Convert label to a number if available
        if claim_label is not None:
            label = self.label_mapping[claim_label]
        else:
            label = -1

        item = {
            "input_ids": torch.tensor(tokens["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(tokens["attention_mask"], dtype=torch.long),
            "claim_id": claim_id,
            "claim_text": claim_text,
            "labels": torch.tensor(label, dtype=torch.long),
            "evidences": evidence_ids
        }

        return item


# Convert labels to numbers
label_mapping = {
    "SUPPORTS": 0,
    "REFUTES": 1,
    "NOT_ENOUGH_INFO": 2,
    "DISPUTED": 3,
}

train_dataset = convertPytorchDataset(train_data, evidence_data, tokenizer, label_mapping)
dev_dataset = convertPytorchDataset(dev_data, evidence_data, tokenizer, label_mapping)

def collate_fn(batch):
    non_empty_batch = [x for x in batch if x is not None]
    if len(non_empty_batch) == 0:
        return None

    max_len = max([x["input_ids"].shape[0] for x in non_empty_batch])
    input_ids = torch.zeros(len(non_empty_batch), max_len, dtype=torch.long)
    attention_mask = torch.zeros(len(non_empty_batch), max_len, dtype=torch.long)
    labels = torch.zeros(len(non_empty_batch), dtype=torch.long)
    claim_ids = []
    claim_texts = []
    evidences = []

    for i, x in enumerate(non_empty_batch):
        cur_len = x["input_ids"].shape[0]
        input_ids[i, :cur_len] = x["input_ids"]
        attention_mask[i, :cur_len] = x["attention_mask"]
        labels[i] = x["labels"]
        claim_ids.append(x["claim_id"])
        claim_texts.append(x["claim_text"])
        evidences.append(x["evidences"])

    return {"input_ids": input_ids, "attention_mask": attention_mask, "claim_id": claim_ids, "claim_text": claim_texts, "labels": labels, "evidences": evidences}



train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
dev_dataloader = DataLoader(dev_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)


# Training the model

Fine-tune hyperparameter

In [ ]:
search_space = {
    'learning_rate': [1e-4, 3e-5, 1e-5],
    'batch_size': [8, 16, 32],
    'epochs': [2, 3, 4]
}

def train_and_evaluate(params):
    batch_size = params['batch_size']
    lr = params['learning_rate']
    epochs = params['epochs']

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

    for epoch in range(epochs):
        train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, device)

    val_accuracy = evaluate(model, dev_dataloader, device)

    return val_accuracy


# import itertools

# best_val_accuracy = 0
# best_params = None

# for lr, batch_size, epochs in itertools.product(search_space['learning_rate'], search_space['batch_size'], search_space['epochs']):
#     params = {'learning_rate': lr, 'batch_size': batch_size, 'epochs': epochs}
#     print(f"Trying parameters: {params}")
#     val_accuracy = train_and_evaluate(params)
#     print(f"Validation Accuracy: {val_accuracy:.4f}")

#     if val_accuracy > best_val_accuracy:
#         best_val_accuracy = val_accuracy
#         best_params = params

# print(f"Best hyperparameters: {best_params}")
# print(f"Best validation accuracy: {best_val_accuracy:.4f}")



In [6]:
from transformers import BertForSequenceClassification

num_labels = len(label_mapping)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

# If gpu available, use gpu.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


from transformers import AdamW, get_linear_schedule_with_warmup

epochs = 3
lr = 3e-5

optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

from tqdm import tqdm

def train_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, device)
    print(f"Train Loss: {train_loss:.4f}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1/3


Training:   0%|          | 0/614 [00:00<?, ?it/s]

Free GPU memory

In [ ]:
import torch

# Clear the GPU cache
torch.cuda.empty_cache()

from tensorflow.keras import backend as K

# Clear the GPU memory
K.clear_session()

# Prediction on dev data

In [ ]:
label_mapping_inverse = {v: k for k, v in label_mapping.items()}

def save_predictions(model, dataloader, output_file):
    model.eval()
    predictions = {}

    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        claim_ids = batch["claim_id"]
        claim_texts = batch["claim_text"]

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            _, predicted = torch.max(outputs.logits, 1)

        for idx, claim_id in enumerate(claim_ids):
            predictions[claim_id] = {
                "claim_text": claim_texts[idx],
                "claim_label": label_mapping_inverse[predicted[idx].item()],
                "evidences": batch["evidences"][idx]
            }

    with open(output_file, "w") as outfile:
        json.dump(predictions, outfile, indent=2)

save_predictions(model, dev_dataloader, "dev_claims_out.json")



# Prediction on test data

In [ ]:
# Prediction function for both dev and test data
def save_predictions(model, dataloader, output_file, has_labels=True):
    model.eval()
    predictions = {}

    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        claim_ids = batch["claim_id"]
        claim_texts = batch["claim_text"]

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            _, predicted = torch.max(outputs.logits, 1)

        for idx, claim_id in enumerate(claim_ids):
            predictions[claim_id] = {
                "claim_text": claim_texts[idx],
                "claim_label": label_mapping_inverse[predicted[idx].item()],
                "evidences": batch["evidences"][idx]
            }

    with open(output_file, "w") as outfile:
        json.dump(predictions, outfile, indent=2)

# Save predictions on dev data
save_predictions(model, dev_dataloader, "dev_claims_out.json")

# Save predictions on test data
test_dataset = convertPytorchDataset(test_data, evidence_data, tokenizer, label_mapping)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)
save_predictions(model, test_dataloader, "test_claims_out.json", has_labels=False)